# IX: The Hermit
### Schrodinger's Cat or Not, classifier test 2
Scope: Test-classifier to see if a neural network can learn how to classify a 3x3 complex matrix as hermitian or not
May 24-25, 2021

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
#@title Build Our Model
#@markdown We use a very simple neural network to keep things straightforward
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape = (3,3,2)), 
                                    tf.keras.layers.Dense(32, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)])
print(model.summary())
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 18)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                608       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 641
Trainable params: 641
Non-trainable params: 0
_________________________________________________________________
None


In [3]:
#@title Some Useful Custom Functions
#@markdown Hermiticity checker, random complex matrix generator

def is_hermitian(M: np.matrix):
    return (M == M.H).all()

def rand_complex(N):
    '''
    Random complex matrix
    '''
    reals = np.random.random((N, N))
    imags = np.random.random((N, N)) * 1j

    return np.matrix(reals + imags)

In [4]:
#@title prefabbed code from SCoN
#@markdown Hermitian generator
def rand_diag(N):
    '''
    N Dimensional random trace-1 diagonal matrix via the following process:
        - Generate a random row vector
        - Divide by the sum to get trace-1
        - Return diagonal matrix
    '''
    diag_elements = np.random.random(N)
    return np.matrix(np.diag(diag_elements/sum(diag_elements)))


def rand_unitary(N):
    '''
    Random complex unitary matrix via the following process:
        - Generate a random complex matrix
        - Apply QR decomposition to get a unitary Q matrix
        - Return Q
    '''
    reals = np.random.random((N, N))
    imags = np.random.random((N, N)) * 1j

    a = reals + imags
    # Run QR decomposition to get unitary operator
    q, _ = np.linalg.qr(a)
    return np.matrix(q)


def rand_hermitian(N):
    '''
    Random trace-1 hermitian matrix of dimension N
    '''
    U = rand_unitary(N)
    H = rand_diag(N)

    return U * H * U.H

In [5]:
#@title Generate the Dataset
#@markdown Training with 100 matrices
matrices = np.zeros((200, 3, 3, 2))
results = np.zeros(200)

for i in range(100):
  
  hermit = True
  while hermit:
    mat = rand_complex(3)
    hermit = is_hermitian(mat)
  matrices[2*i,:,:,0] = mat.real
  matrices[2*i,:,:,1] = mat.imag
  results[2*i] = 0

  H = rand_hermitian(3)
  matrices[2*i + 1,:,:,0] = H.real
  matrices[2*i + 1,:,:,1] = H.imag
  results[2*i + 1] = 1


In [6]:
history = model.fit(x=matrices, y=results, epochs=10, steps_per_epoch=100, verbose=1)

Epoch 1/10
100/100 [==============================] - 1s 1ms/step - loss: 0.7115 - accuracy: 0.5662
Epoch 2/10
100/100 [==============================] - 0s 1ms/step - loss: 0.3661 - accuracy: 0.9747
Epoch 3/10
100/100 [==============================] - 0s 1ms/step - loss: 0.2118 - accuracy: 1.0000
Epoch 4/10
100/100 [==============================] - 0s 1ms/step - loss: 0.1078 - accuracy: 1.0000
Epoch 5/10
100/100 [==============================] - 0s 1ms/step - loss: 0.0679 - accuracy: 1.0000
Epoch 6/10
100/100 [==============================] - 0s 1ms/step - loss: 0.0400 - accuracy: 1.0000
Epoch 7/10
100/100 [==============================] - 0s 1ms/step - loss: 0.0261 - accuracy: 1.0000
Epoch 8/10
100/100 [==============================] - 0s 1ms/step - loss: 0.0226 - accuracy: 1.0000
Epoch 9/10
100/100 [==============================] - 0s 1ms/step - loss: 0.0135 - accuracy: 1.0000
Epoch 10/10
100/100 [==============================] - 0s 1ms/step - loss: 0.0109 - accuracy: 1.0000

In [10]:
#@title Validate our results
#@markdown Generate 1 Hermitian and 1 random (probably non-hermitian) matrix to test!

mat = rand_hermitian(3)
bad = rand_complex(3)

bruh = np.empty((2, 3, 3, 2))

bruh[0,:,:,0] = mat.real
bruh[0,:,:,1] = mat.imag

bruh[1,:,:,0] = bad.real
bruh[1,:,:,1] = bad.imag


model.predict(bruh)

array([[0.9909215 ],
       [0.00399008]], dtype=float32)

# Conclusion
This appears to work!